In [8]:
import pandas as pd
import numpy as np
data = pd.read_csv('test_data.csv', header=None)
array = data.values
array

array([[11111],
       [19991],
       [19191],
       [19991],
       [11111]])

In [10]:

numbers = [str(no[0]) for no in data.values.tolist()]

In [19]:
numbers = [str(no[0]) for no in data.values.tolist()]
original_array = []
for idx in range(len(numbers)):
    original_array.append([int(i) for i in numbers[idx]])
np.array(original_array) + 1

array([[ 2,  2,  2,  2,  2],
       [ 2, 10, 10, 10,  2],
       [ 2, 10,  2, 10,  2],
       [ 2, 10, 10, 10,  2],
       [ 2,  2,  2,  2,  2]])

In [38]:
kernel = np.array([[1,1,1],[1,0,10],[1,10,10]])
kernel2 = np.array([[1,1,1],[1,0,1],[1,1,1]])
np.where(kernel != 10 & 0)

(array([0, 0, 0, 1, 1, 2, 2, 2]), array([0, 1, 2, 0, 2, 0, 1, 2]))

In [35]:
kernel

array([[ 1,  1,  1],
       [ 1,  0, 10],
       [ 1, 10, 10]])

In [47]:
np.where((kernel < 9) & (kernel != 0), kernel + 1, kernel)

array([[ 2,  2,  2],
       [ 2,  0, 10],
       [ 2, 10, 10]])